## New analysis goal: fixed skill sets from tourneys

Use df_matches to identify the group ids that use competitive rulesets.
This gives us the team ids we need.
Then we can get the team roster using https://fumbbl.com/api/team/get/1102662
we need the player ids here.

and the chosen skills using https://fumbbl.com/api/team/getOptions/1102662
this is a string of player ids, with skill ids.

through https://fumbbl.com/api/skill/list we can get a list of skills.
https://fumbbl.com/api/skill/list/2020 and this is the 2020 list.

* World cup training: 9941 (dec 2020), 2 matches in dec 2020 using bb2016, then in okt 2021 for real. 429 matches.
* SUper league: 15615
* templars road to WC: 11605
* entrainment tournois: 12879
* NAF online tournaments : 9298
* Tacklezone: 12013
* Doppelbock: 13198
* Eurobowl practice league: 15643
* Eurobowl 2020 training: 12087 (eurobowl warsaw)

In [ ]:
import pandas as pd
import numpy as np
import plotnine as p9
import requests

from mizani.formatters import date_format

# point this to the location of the HDF5 datasets
path_to_datasets = 'datasets/current/'

# FUMBBL matches
target = 'df_matches.h5'
df_matches = pd.read_hdf(path_to_datasets + target) 



In [ ]:
# FUMBBL inducements
target = 'inducements.h5'
inducements = pd.read_hdf(path_to_datasets + target) 

In [ ]:
#df_matches.query('match_id == 4257923')['team1_id']
df_matches.query('league == 12013 & team1_value < 1000')


In [ ]:
inducements.query('match_id == 4335359')

In [ ]:

%run read_json_file.py
%run write_json_file.py

# grab BB2016 and BB2020 skills lists from FUMBBL

In [ ]:
api_string = "https://fumbbl.com/api/skill/list"

skill_list = requests.get(api_string)
# PM here save JSON as file
skill_list = skill_list.json()

write_json_file(skill_list, fname = "cache/skill_list_bb2016.json")

In [ ]:
api_string = "https://fumbbl.com/api/skill/list/2020"

skill_list = requests.get(api_string)
# PM here save JSON as file
skill_list = skill_list.json()

write_json_file(skill_list, fname = "cache/skill_list_bb2020.json")

In [ ]:
skill_list = read_json_file(fname = "cache/skill_list_bb2020.json")
skill_list = pd.json_normalize(skill_list)
skill_list.rename({'id' : 'skill_id'}, axis = 1, inplace=True)

skill_list = skill_list.loc[:, ['skill_id', 'name']]

skill_list.head(5)

In [ ]:
skill_list2 = read_json_file(fname = "cache/skill_list_bb2016.json")
skill_list2 = pd.json_normalize(skill_list2)
skill_list2.rename({'id' : 'skill_id'}, axis = 1, inplace=True)

skill_list2 = skill_list2.loc[:, ['skill_id', 'name']]

skill_list2.head(5)

In [ ]:
skill_list = pd.concat([skill_list, skill_list2])

In [ ]:
skill_list

# Gather Team roster info

Need a combi of Team and Match to get the inducements for the roster

In [ ]:
df_matches.query('tournament_id == 58323')['match_id'].max()

In [ ]:
df_matches.query('match_id == 4418354')['team2_id']

In [ ]:
match_id, team_id = [4337210, 1050197] # 1150
match_id, team_id = [4261575, 1000657] # 880 no skills + SP
match_id, team_id = [4261575, 1000538] # 780no skills + SP
match_id, team_id = [4335316, 1050180] # 875 w skills + SP
match_id, team_id = [4339241,1052367] # 810 w skills + SP
match_id, team_id = [4377476,1075377] # 925 w skills + SP	
match_id, team_id = [4378237, 1075893] # 820 w skills, SP and bribe
match_id, team_id = [4381569, 1078092]
match_id, team_id = [4418354, 1093446]
match_id, team_id = [4418354, 1094174]
match_id, team_id = [4246968, 1006235] # 2020 too old no skill in team yet
match_id, team_id = [4369687, 1067847] # 2022 has skills
match_id, team_id = [4343656, 1046931] # nov 2021 too old
match_id, team_id = [4243298, 997766] #   2020 eurobowl warsaw practice, bb2016, heeft 2016 skill list nodig


https://fumbbl.com/api/ruleset/get/188
contains info about gold and skills.

From the roster id we can infer the bb version.

In [ ]:
import gzip

dirname = "raw/team_files/" + str(team_id)[0:4]

fname_string_gz = dirname + "/team_" + str(team_id) + ".json.gz"        

# PM read compressed json file
with gzip.open(fname_string_gz, mode = "rb") as f:
    team_obj = json.load(f)

fname_string_gz = dirname + "/team_" + str(team_id) + "_skills.json.gz"        

# PM read compressed json file
with gzip.open(fname_string_gz, mode = "rb") as f:
    team_skill_obj = json.load(f)


Next we extract the player info into a nice pd dataFrame:

In [ ]:
df_roster = pd.json_normalize(team_obj, 'players', ['rerolls', 'assistantCoaches', 'cheerleaders', 'apothecary', ['roster', 'name']])

df_roster.rename({'id' : 'player_id'}, axis = 1, inplace = True)

df_roster = df_roster.loc[:, ['player_id', 'number', 'position', 'rerolls', 'assistantCoaches', 'cheerleaders', 'apothecary', 'roster.name']]

df_roster['team_id'] = team_id

In [ ]:
team_skill_obj

pd_skills = pd.json_normalize(team_skill_obj, 'tournamentSkills')

pd_skills.info()

next we add the skill info to the roster.

In [ ]:
pd_skills = pd.json_normalize(team_skill_obj, 'tournamentSkills')

if pd_skills.empty is False:    
    pd_skills.columns = ['player_id', 'skill_id']

    pd_skills['player_id'] = pd_skills['player_id'].fillna(0).astype(np.int64)
    pd_skills['skill_id'] = pd_skills['skill_id'].fillna(0).astype(np.int64)

else:
    pd_skills = pd.DataFrame({'player_id': [-1], 'skill_id': [-1]})

# Next we add the skill names:
pd_skills= pd.merge(pd_skills, skill_list, on='skill_id', how='left')
pd_skills

next we combine the skills with the roster

In [ ]:

df_roster = pd.merge(df_roster, pd_skills, on='player_id', how='left')


Next we add the inducements

In [ ]:
if (df_matches.query('match_id == @match_id')['team1_id'].values[0] == team_id):
    inducement_id = 'team1'
elif (df_matches.query('match_id == @match_id')['team2_id'].values[0] == team_id):
    inducement_id = 'team2'
else:
    inducement_id = -1

inducements_to_add = inducements.query('match_id == @match_id and team == @inducement_id')['inducements'].values.tolist()



inducements_to_add

In [ ]:
for inducement in inducements_to_add:
    if not inducement == ['']:
        new_row = df_roster.iloc[[1]].copy()
        new_row['position'] = inducement
        new_row['player_id'] = 99999999
        new_row['number'] = 99
        new_row['skill_id'] = np.NaN
        new_row['name'] = np.nan
        df_roster = df_roster.append(new_row)

df_roster